In [14]:
from tensorflow import keras
from keras import layers
import os
import numpy as np
import matplotlib

In [15]:
original_dim = (256,256,3)
intermediate_dim = 64
latent_dim = 40

In [44]:
from keras import backend as K

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=0.1)
    return z_mean + K.exp(z_log_sigma) * epsilon

inputs = keras.Input(shape=original_dim)

# Create/Apply Conv Layers
bn1 = layers.BatchNormalization()(inputs)
c1 = layers.Conv2D(32, (2,2), activation='relu', padding='same')(bn1)
mp1 = layers.MaxPooling2D((4, 4), padding='same')(c1)
dpo1 = layers.Dropout(0.3)(mp1)

bn2 = layers.BatchNormalization()(dpo1)
c2 = layers.Conv2D(16, (4,4), activation='relu', padding='same')(bn2)
mp2 = layers.MaxPooling2D((4, 4), padding='same')(c2)
dpo2 = layers.Dropout(0.3)(mp2)

bn3 = layers.BatchNormalization()(dpo2)
c3 = layers.Conv2D(8, (8,8), activation='relu', padding='same')(bn3)
mp3 = layers.MaxPooling2D((2, 2), padding='same')(c3)
dpo3 = layers.Dropout(0.3)(mp3)
print(dpo3.shape)

# Flatten the layer
flat = layers.Flatten()(dpo3)
print(flat.shape)
h = layers.Dense(intermediate_dim, activation='relu')(flat)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)
z = layers.Lambda(sampling)([z_mean, z_log_sigma])

# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
x = layers.Dense(512, activation='relu')(x)

reshape = layers.Reshape((8,8,8))(x) # undo flatten
# Deconvolution
bn4 = layers.BatchNormalization()(reshape)
c4 = layers.Conv2D(8, (8,8), activation='relu', padding='same')(bn4)
us4 = layers.UpSampling2D((4, 4))(c4)
dpo4 = layers.Dropout(0.3)(us4)

bn5 = layers.BatchNormalization()(dpo4)
c5 = layers.Conv2D(16, (4,4), activation='relu', padding='same')(bn5)
us5 = layers.UpSampling2D((2, 2))(c5)
dpo5 = layers.Dropout(0.3)(us5)

bn6 = layers.BatchNormalization()(dpo5)
c6 = layers.Conv2D(32, (2,2), activation='relu', padding='same')(bn6)
us6 = layers.UpSampling2D((4, 4))(c6)
dpo6 = layers.Dropout(0.3)(us6)

# bn7 = layers.BatchNormalization()(dpo6)
# c7 = layers.Conv2D(32, (2,2), activation='relu', padding='same')(bn7)
# us7 = layers.UpSampling2D((4, 4))(c7)
# dpo7 = layers.Dropout(0.3)(us7)

# bn8 = layers.BatchNormalization()(dpo7)
# c8 = layers.Conv2D(32, (2,2), activation='relu', padding='same')(bn8)
# us8 = layers.UpSampling2D((4, 4))(c8)
# dpo8 = layers.Dropout(0.3)(us8)

outputs = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(dpo6)


# outputs = layers.Dense(original_dim[1], activation='sigmoid')()






decoder = keras.Model(latent_inputs, outputs, name='decoder')





# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')


(None, 8, 8, 8)
(None, 512)


In [45]:
# Create the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim[1]
kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
print(vae.summary())

Model: "vae_mlp"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 encoder (Functional)           [(None, 40),         55060       ['input_5[0][0]']                
                                 (None, 40),                                                      
                                 (None, 40)]                                                      
                                                                                                  
 decoder (Functional)           (None, 256, 256, 3)  45147       ['encoder[0][2]']          

In [46]:
from PIL import Image
subreddits = os.listdir("D:\MPH-2022-Data")
for subreddit in subreddits:
    subreddit_folder = os.path.join("D:\MPH-2022-Data", subreddit)
    image_paths = [os.path.join(subreddit_folder, filename) for filename in os.listdir(subreddit_folder) if ".csv" not in filename]
    for image_path in image_paths:
        image = Image.open(image_path)
        w = image.width
        h = image.height
        crop = min(w, h)
        crop_dim = (w//2 - crop//2, h//2 - crop//2, w//2 + crop//2, h//2 + crop//2)
        image = image.crop(crop_dim)
        image.thumbnail(original_dim[:2])
        # image.show()
        # image_array = np.asarray(image)
        # print(image_array.shape)
        

KeyboardInterrupt: 

In [55]:
from keras.utils import Sequence
import numpy as np
import pandas as pd
class Custom_Generator(Sequence):

    #initialise generator
    def __init__(self, data_folder_path, batch_size) :
        self.data_folder_path = data_folder_path
        self.batch_size = batch_size
        self.total_number_of_files = 0
        self.number_of_files = {}
        for subreddit_folder in os.listdir(data_folder_path):
            subreddit_folder_path = os.path.join(data_folder_path, subreddit_folder)
            # print(subreddit_folder)
            try:
                data = pd.read_csv(os.path.join(subreddit_folder_path, f"{subreddit_folder}.csv"), delimiter=";")
            except:
                continue
            data = data[data["Type"] == "image"]
            self.number_of_files[subreddit_folder] = len(data)
        self.total_number_of_files = sum(self.number_of_files.values())
    
    #returns number of data batches this generator will return
    def __len__(self) :
        return np.ceil(self.total_number_of_files / float(self.batch_size)).astype(np.int)
  
    def preprocess_image(self, image_path):
        image = Image.open(image_path)
        w = image.width
        h = image.height
        crop = min(w, h)
        crop_dim = (w//2 - crop//2, h//2 - crop//2, w//2 + crop//2, h//2 + crop//2)
        image = image.crop(crop_dim)
        image.thumbnail(original_dim[:2])    
        image_array = np.asarray(image)
        # print(image_array.shape)
        if len(image_array.shape) == 4: 
            return image_array[:,:,:,0]
        else: 
            return image_array

    #returns a single batch 
    def __getitem__(self, idx) :
        subreddit_folders = self.number_of_files.keys()
        images = []
        idx = 0
        for subreddit_folder in subreddit_folders:
            # Get the image paths for this subreddit folder
            subreddit_folder_path = os.path.join(self.data_folder_path, subreddit_folder)
            data = pd.read_csv(os.path.join(subreddit_folder_path, f"{subreddit_folder}.csv"), delimiter=";")
            data = data[data["Type"] == "image"]
            image_paths = data["Content File"]
            # If we can't add all of them, add while we can
            if len(images) + self.number_of_files[subreddit_folder] > self.batch_size:
                idx = 0
                while len(images) < self.batch_size:
                    images.append(self.preprocess_image(image_paths[idx]))
                    idx += 1
            # Otherwise, add them all
            else:
                images.extend([self.preprocess_image(image_path) for image_path in image_paths])

        return (np.array(images), np.array(images))

In [57]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
history = vae.fit(
    Custom_Generator("D:\MPH-2022-Data", 1),
    epochs=10,
)

Epoch 1/10


c:\Users\Alex\Development-Tools\Python-Environments\MPH-2022-Env\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
C:\Users\Alex\AppData\Local\Temp\ipykernel_31612\1190475767.py:25: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.ceil(self.total_number_of_files 

  410/24111 [..............................] - ETA: 3:39:51 - loss: -75894544.0000

KeyboardInterrupt: 